In [1]:
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import requests
global pages

In [2]:

my_URL = "https://www.statbunker.com/competitions/LeagueTable?comp_id=13"
html = requests.get(my_URL).text
soup = BeautifulSoup(html, "html.parser")

In [3]:
for i in soup.find_all("div", attrs={"id":"right"}):
    items = i.select("option[value]")
    values=[item.get('value') for item in items]
values = [s for s in values if s.isdigit()]
values.reverse()

In [4]:
#Get Table Column Headers
song_link = soup.find("thead")
list_header=[]
x=0
Season_Year_list =[]
for i in song_link.find_all("th"):
    if(x<2 or x==9):
        list_header.append(i.get_text())
    elif(x<10):
        list_header.append(i.get('title'))
    x+=1
list_header.append('Year') #add current year
    
df = pd.DataFrame(columns=list_header)

#Set The Whole DataFrame (All Seasons together) , Running on all pages
for i in values:
    my_URL = "https://www.statbunker.com/competitions/LeagueTable?comp_id="+i
    html = requests.get(my_URL).text
    soup = BeautifulSoup(html, "html.parser")

#Check Season Year
    for i in soup.find_all(itemprop="title"):
        Season_Year=i.get_text()
    Season_Year= Season_Year[len(Season_Year)-5:len(Season_Year)]
    Season_Year_list.append(Season_Year)

    
#Get Table Team Data
    song_team = soup.find("tbody")
    list_team =[]
    z=0
    
    for i in song_team.find_all("td"):
        if (i.get_text()!= 'More'):
            list_team.append(i.get_text())
        #print(z,i.get_text()) Test
    
        z+=1
    
        if(z>9):
            list_team.append(Season_Year)
            df_length = len(df)
            df.loc[df_length] = list_team
            z=-1
            list_team=[]


display(df)
#https://www.statbunker.com/competitions/LeagueTable?comp_id=667 # 20/21 last season
#https://www.statbunker.com/competitions/LeagueTable?comp_id=13  # 92/93 first season



,#,Clubs,Matches played,Matches won,Matches drawn,Matches lost,Goals for,Goals against,Goal difference,Points,Year
0,1,Manchester United,42,24,12,6,67,31,36,84,92/93
1,2,Aston Villa,42,21,11,10,57,40,17,74,92/93
2,3,Norwich City,42,21,9,12,61,65,-4,72,92/93
3,4,Blackburn Rovers,42,20,11,11,68,46,22,71,92/93
4,5,Queens Park Rangers,42,17,12,13,63,55,8,63,92/93
...,...,...,...,...,...,...,...,...,...,...,...
581,16,Brighton & Hove Albion,24,5,11,8,25,30,-5,26,20/21
582,17,Newcastle United,24,7,4,13,25,40,-15,25,20/21
583,18,Fulham,24,3,10,11,20,32,-12,19,20/21
584,19,West Bromwich Albion,24,2,7,15,19,55,-36,13,20/21
